In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

ratings = pd.read_csv('/content/drive/MyDrive/IR/ratings.csv', sep='\t', encoding='latin-1',
                      usecols=['user_id', 'movie_id', 'user_emb_id', 'movie_emb_id', 'rating'])

movies = pd.read_csv('/content/drive/MyDrive/IR/movies.csv', sep='\t', encoding='latin-1',
                     usecols=['movie_id', 'title', 'genres'])

In [3]:
movies.head(5)

,movie_id,title,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [4]:
ratings.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354


### Content Based

**Objective**: Build a Content-Based Recommendation system that computes similarity between movies based on movie genres. It will suggest movies that are most similar to a particular movie based on its genre.

**Dataset**:
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Split the `genres` column into separate genre strings.
   - Fill any missing values in the `genres` column and convert it to string.

2. **Compute TF-IDF Matrix**:
   - Use `TfidfVectorizer` to transform the genres into a TF-IDF matrix.

3. **Calculate Cosine Similarity**:
   - Compute cosine similarity between the TF-IDF matrices of movies.

4. **Recommend Movies**:
   - Create a function that gets movie recommendations based on the cosine similarity score of movie genres.

In [5]:
movies['genres'] = movies['genres'].str.split('|')
movies['genres'] = movies['genres'].fillna("").astype('str')

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])

In [7]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [8]:
def genre_recommendations(title, n=10):
    #TODO
    index = movies[movies['title'] == title].index[0]
    similarity_scores = []

    for i, score in enumerate(cosine_sim[index]):
        similarity_scores.append((i, score))

    for i in range(len(similarity_scores)):
        for j in range(i + 1, len(similarity_scores)):
            if similarity_scores[i][1] < similarity_scores[j][1]:
                similarity_scores[i], similarity_scores[j] = similarity_scores[j], similarity_scores[i]

    similarity_scores = similarity_scores[1:n+1]
    recommended_indices = [i for i, _ in similarity_scores]
    return movies.loc[recommended_indices, 'title']

In [9]:
# Example
genre_recommendations('GoldenEye (1995)')

345     Clear and Present Danger (1994)
543           Surviving the Game (1994)
724                    Rock, The (1996)
788                     Daylight (1996)
825               Chain Reaction (1996)
978                 Maximum Risk (1996)
1467                    Anaconda (1997)
1513                     Con Air (1997)
1693                   Firestorm (1998)
3686          Perfect Storm, The (2000)
Name: title, dtype: object


### Collaborative Filtering

**Objective**: Implement a collaborative filtering movie recommendation system to recommend movies to a user based on similar users' spreferences.

**Dataset**:

- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Split `genres` column into separate rows.

2. **Create User-Genre Matrix**:
   - Create a matrix where rows are users and columns are genres.
   - Each cell represents the ratio of movies watched by the user in that genre to the total movies watched by the user.

3. **Calculate User Similarity**:
   - Use cosine similarity to calculate the similarity between users.

4. **Find Top N Similar Users**:
   - Find the top N users with the most similar preferences to the given user.

5. **Recommend Movies**:
   - Recommend movies that similar users have watched but the target user has not.
   - Return titles and genres of the top 10 recommended movies.

In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
data = pd.merge(ratings, movies, on='movie_id')

In [12]:
data.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
1,2,1193,5,1,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
2,12,1193,4,11,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
3,15,1193,4,14,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
4,17,1193,5,16,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']


In [13]:
data = data.assign(genres=data['genres'].str.split('|')).explode('genres')

In [14]:
data.head(5)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
0,1,1193,5,0,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
1,2,1193,5,1,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
2,12,1193,4,11,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
3,15,1193,4,14,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']
4,17,1193,5,16,1192,One Flew Over the Cuckoo's Nest (1975),['Drama']


In [46]:
def get_top_n_similar_users(user_id, n=5):
    #TODO
    uid = user_id
    top_n = n

    grouped_data = data.groupby('user_id')['genres'].apply(list)
    user_profiles = pd.get_dummies(grouped_data.apply(pd.Series).stack()).groupby(level=0).sum()
    user_profiles = user_profiles.fillna(0)

    all_genres = np.unique(data['genres'])
    genre_to_index = {}
    for index in range(len(all_genres)):
        genre_to_index[all_genres[index]] = index

    profile_matrix = np.zeros((len(user_profiles), len(all_genres)))
    for profile_id, profile in enumerate(user_profiles):
        for genre_name in user_profiles.columns:
            genre_score = user_profiles.iloc[profile_id][genre_name]
            profile_matrix[profile_id, genre_to_index[genre_name]] = genre_score

    current_user_profile = profile_matrix[uid-1]
    similarity_scores = cosine_similarity(profile_matrix, [current_user_profile])

    sorted_indices = []
    for idx in range(len(similarity_scores)):
        sorted_indices.append((idx, similarity_scores[idx][0]))
    sorted_indices.sort(key=lambda x: x[1], reverse=True)

    top_similar_users = []
    for similar_idx in range(1, top_n+1):
        top_similar_users.append(sorted_indices[similar_idx][0])

    return top_similar_users, similarity_scores[top_similar_users]

In [44]:
def recommend_movies(user_id, n_similar_users=5, n_recommendations=10):
    #TODO
    similar_user_ids, similarity_scores = get_top_n_similar_users(user_id, n_similar_users)

    current_user_data = data[data['user_id'] == user_id]
    watched_movies = set(current_user_data['title'])

    movie_recommendations = []
    movie_frequency = {}

    for idx in range(len(similar_user_ids)):
        matched_user_data = data[data['user_id'] == similar_user_ids[idx] + 1]
        matched_user_movies = set(matched_user_data['title'])

        for film in matched_user_movies:
            if film not in watched_movies:
                if film not in movie_frequency:
                    movie_frequency[film] = 0
                movie_frequency[film] += similarity_scores[idx][0]

    sorted_recommendations = []
    for item in movie_frequency.items():
        sorted_recommendations.append(item)
    sorted_recommendations.sort(key=lambda x: x[1], reverse=True)

    for movie_index in range(n_recommendations):
        movie, _ = sorted_recommendations[movie_index]
        movie_recommendations.append(movie)

    final_recommendations = pd.merge(movies, pd.DataFrame(movie_recommendations, columns=['title']), on='title')
    return final_recommendations.head(n_recommendations)


In [47]:
# Example
user_id = 1
recommended_movies = recommend_movies(user_id, n_similar_users=5, n_recommendations=10)

In [48]:
recommended_movies

,movie_id,title,genres
0,162,Crumb (1994),['Documentary']
1,296,Pulp Fiction (1994),"['Crime', 'Drama']"
2,593,"Silence of the Lambs, The (1991)","['Drama', 'Thriller']"
3,994,Big Night (1996),['Drama']
4,1280,Raise the Red Lantern (1991),['Drama']
5,1617,L.A. Confidential (1997),"['Crime', 'Film-Noir', 'Mystery', 'Thriller']"
6,2599,Election (1999),['Comedy']
7,2858,American Beauty (1999),"['Comedy', 'Drama']"
8,2966,"Straight Story, The (1999)",['Drama']
9,2997,Being John Malkovich (1999),['Comedy']


### SVD (Singular Value Decomposition)



**Objective**: Implement an SVD-based recommendation system to recommend movies to users by decomposing the user-item interaction matrix into latent factors.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Preprocess Data**:
   - Merge `ratings` and `movies` DataFrames on `movie_id`.
   - Create a user-item matrix where rows represent users, columns represent movies, and the values are the ratings.

2. **Decompose Matrix using SVD**:
   - Apply Singular Value Decomposition (SVD) to decompose the user-item matrix into three matrices: $U$, $\Sigma$, and $V^T$.

3. **Reconstruct Matrix**:
   - Reconstruct the user-item matrix using the top $k$ singular values to reduce dimensionality.

4. **Predict Ratings**:
   - Use the reconstructed matrix to predict ratings for all user-item pairs.

5. **Recommend Movies**:
   - Recommend the top 20 movies with the highest predicted ratings for a given user that the user hasn't rated yet.

In [49]:
n_users = ratings.user_id.unique().shape[0]
n_movies = ratings.movie_id.unique().shape[0]
print('Number of users = ' + str(n_users) + ' | Number of movies = ' + str(n_movies))

Number of users = 6040 | Number of movies = 3706


Fill na elements.

In [50]:
Ratings = ratings.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
Ratings.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [54]:
# calculate rating matrix
R = Ratings.to_numpy()

ratings_mean = np.empty(R.shape[0])

for i in range(R.shape[0]):
    ratings_mean[i] = np.mean(R[i])

ratings_here = R - ratings_mean.reshape(-1, 1)

In [53]:
from scipy.sparse.linalg import svds

U, Sigma, V_transposed = svds(ratings_here, k=50)
Sigma = np.diag(Sigma)

In [55]:
all_user_predicted_ratings = np.dot(np.dot(U, Sigma), V_transposed) + ratings_mean.reshape(-1, 1)

In [56]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = Ratings.columns)
preds.head()

movie_id,1,2,3,4,5,6,7,8,9,10,...,3943,3944,3945,3946,3947,3948,3949,3950,3951,3952
0,4.288861,0.143055,-0.195080,-0.018843,0.012232,-0.176604,-0.074120,0.141358,-0.059553,-0.195950,...,0.027807,0.001640,0.026395,-0.022024,-0.085415,0.403529,0.105579,0.031912,0.050450,0.088910
1,0.744716,0.169659,0.335418,0.000758,0.022475,1.353050,0.051426,0.071258,0.161601,1.567246,...,-0.056502,-0.013733,-0.010580,0.062576,-0.016248,0.155790,-0.418737,-0.101102,-0.054098,-0.140188
2,1.818824,0.456136,0.090978,-0.043037,-0.025694,-0.158617,-0.131778,0.098977,0.030551,0.735470,...,0.040481,-0.005301,0.012832,0.029349,0.020866,0.121532,0.076205,0.012345,0.015148,-0.109956
3,0.408057,-0.072960,0.039642,0.089363,0.041950,0.237753,-0.049426,0.009467,0.045469,-0.111370,...,0.008571,-0.005425,-0.008500,-0.003417,-0.083982,0.094512,0.057557,-0.026050,0.014841,-0.034224
4,1.574272,0.021239,-0.051300,0.246884,-0.032406,1.552281,-0.199630,-0.014920,-0.060498,0.450512,...,0.110151,0.046010,0.006934,-0.015940,-0.050080,-0.052539,0.507189,0.033830,0.125706,0.199244


In [57]:
def recommend_movies(predictions, userID, movies, original_ratings, num_recommendations):

    user_row_number = userID - 1 # User ID starts at 1, not 0
    user_index = user_row_number

    sorted_user_predictions = predictions.iloc[user_index].sort_values(ascending=False)

    user_data = original_ratings[original_ratings.user_id == (userID)]

    # merfe movies data with user_data
    user_full = user_data.merge(movies, on='movie_id', how='left').sort_values(by='rating', ascending=False)



    print(f'User {userID} has already rated {user_full.shape[0]} movies.')
    print(f'Recommending highest {num_recommendations} predicted ratings movies not already rated.')

    recommendations = (
        movies[~ movies['movie_id'].isin(user_full['movie_id'])]
        .merge(pd.DataFrame(sorted_user_predictions).reset_index(), on='movie_id', how='left')
        .rename(columns={user_index: 'PredictedRating'})
        .sort_values(by='PredictedRating', ascending=False)
        .iloc[:num_recommendations, :-1]
    )

    return user_full, recommendations

In [58]:
already_rated, predictions = recommend_movies(preds, 4375, movies, ratings, 20)

User 4375 has already rated 325 movies.
Recommending highest 20 predicted ratings movies not already rated.


In [59]:
ratings.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id
0,1,1193,5,0,1192
1,1,661,3,0,660
2,1,914,3,0,913
3,1,3408,4,0,3407
4,1,2355,5,0,2354
5,1,1197,3,0,1196
6,1,1287,5,0,1286
7,1,2804,5,0,2803
8,1,594,4,0,593
9,1,919,4,0,918


In [60]:
# Top 20 movies that User 4375 has rated
already_rated.head(20)

,user_id,movie_id,rating,user_emb_id,movie_emb_id,title,genres
304,4375,2797,5,4374,2796,Big (1988),"['Comedy', 'Fantasy']"
127,4375,1961,5,4374,1960,Rain Man (1988),['Drama']
125,4375,349,5,4374,348,Clear and Present Danger (1994),"['Action', 'Adventure', 'Thriller']"
109,4375,2533,5,4374,2532,Escape from the Planet of the Apes (1971),"['Action', 'Sci-Fi']"
122,4375,3552,5,4374,3551,Caddyshack (1980),['Comedy']
121,4375,14,5,4374,13,Nixon (1995),['Drama']
307,4375,527,5,4374,526,Schindler's List (1993),"['Drama', 'War']"
119,4375,318,5,4374,317,"Shawshank Redemption, The (1994)",['Drama']
118,4375,1923,5,4374,1922,There's Something About Mary (1998),['Comedy']
309,4375,2004,5,4374,2003,Gremlins 2: The New Batch (1990),"['Comedy', 'Horror']"


#### Evaluation

In [61]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt

train_data, test_data = train_test_split(ratings, test_size=0.2)

train_ratings = train_data.pivot(index = 'user_id', columns ='movie_id', values = 'rating').fillna(0)
R_train = train_ratings.values

# user_ratings_mean_train = # TODO
# Ratings_demeaned_train = R_train - user_ratings_mean_train.reshape(-1, 1)

U_train, Sigma_train, V_transposed_train = svds(R_train, k=50)
Sigma_train = np.diag(Sigma_train)

all_user_predicted_ratings_train = np.dot(np.dot(U_train, Sigma_train), V_transposed_train)
preds_train = pd.DataFrame(all_user_predicted_ratings_train, columns=train_ratings.columns)

def predict_rating(user_id, movie_id):
    # TODO
    user_index = user_id - 1
    movie_index = movie_id - 1

    if user_index >= preds_train.shape[0] or movie_index >= preds_train.shape[1]:

        return np.nan

    predicted_rating = preds_train.iat[user_index, movie_index]

    return predicted_rating

test_data['predicted_rating'] = test_data.apply(lambda x: predict_rating(x['user_id'], x['movie_id']), axis=1)

test_data.dropna(inplace=True)

rmse = sqrt(mean_squared_error(test_data['rating'], test_data['predicted_rating']))
print(f'Root Mean Squared Error: {rmse}')


Root Mean Squared Error: 3.518518833190768


### Nueral Network Model (Recommender Model)



**Objective**: Implement a Recommender model to recommend movies to a user based on similar users' preferences.

**Dataset**:
- **Ratings Data**: DataFrame with columns `user_id`, `movie_id`, and `rating`.
- **Movies Data**: DataFrame with columns `movie_id`, `title`, and `genres`.

**Steps**:

1. **Define Dataset and DataLoader**:
   - Create a custom PyTorch `Dataset` for ratings.
   - Create a DataLoader for batching and shuffling data.

2. **Define the Neural Network**:
   - Create a neural network with embedding layers for users and movies.

3. **Train the Model**:
   - Train the model using Mean Squared Error loss and Adam optimizer.
   - Save model checkpoints.

4. **Evaluate the Model**:
   - Calculate RMSE on the entire dataset.

5. **Predict Ratings for Unrated Movies**:
   - Predict and recommend top 10 unrated movies for a given user.

In [62]:
from torch.utils.data import Dataset, DataLoader
import torch
import torch.nn as nn
from tqdm.notebook import tqdm

In [64]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Define the dataset
class RatingsDataset(Dataset):
    def __init__(self, ratings):
        #TODO
        self.ratings = torch.tensor(ratings['rating'].values, dtype=torch.float32).to(device)
        self.movie_ids = torch.tensor(ratings['movie_id'].values, dtype=torch.long).to(device)
        self.user_ids = torch.tensor(ratings['user_id'].values, dtype=torch.long).to(device)

    def __len__(self):
        #TODO
        return len(self.ratings)

    def __getitem__(self, idx):
        #TODO
        return self.user_ids[idx], self.movie_ids[idx], self.ratings[idx]

# Define the neural network
class RecommenderNet(nn.Module):
    def __init__(self, num_users, num_movies, embedding_size=50):
        super(RecommenderNet, self).__init__()
        #TODO
        self.movie_embedding = nn.Embedding(num_movies, embedding_size)
        self.user_embedding = nn.Embedding(num_users, embedding_size)
        self.fc1 = nn.Linear(embedding_size * 2, 128)
        self.ac1 = nn.ReLU()
        self.fc2 = nn.Linear(128, 128)
        self.ac2 = nn.ReLU()
        self.fc3 = nn.Linear(128, 1)
        self.dropout = nn.Dropout(0.2)

    def forward(self, user, movie):
        #TODO
        user_embedding = self.user_embedding(user)
        movie_embedding = self.movie_embedding(movie)
        x = torch.cat([user_embedding, movie_embedding], dim=1)
        x = self.fc1(x)
        x = self.ac1(x)
        x = self.fc2(x)
        x = self.ac2(x)
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Create the dataset and dataloader

dataset = RatingsDataset(ratings)
dataloader = DataLoader(dataset, batch_size=512, shuffle=True)

max_user_id = -1
max_movie_id = -1

for _, row in ratings.iterrows():
    if row['user_id'] > max_user_id:
        max_user_id = row['user_id']
    if row['movie_id'] > max_movie_id:
        max_movie_id = row['movie_id']

num_users = max_user_id
num_items = max_movie_id


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Initialize the model, loss function and optimizer

model = RecommenderNet(num_users, num_items, embedding_size=50).to(device)
loss_function = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


# Training loop

epochs = 12
for epoch in tqdm(range(epochs)):
    model.train()
    total_loss = 0
    for batch_idx, (user, item, rating) in enumerate(dataloader):
        optimizer.zero_grad()
        prediction = model(user - 1, item - 1)
        loss = loss_function(prediction, rating.view(-1, 1))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f'Epoch {epoch + 1}/{epochs}, Avg Loss: {total_loss / len(dataloader):.4f}')

torch.save(model.state_dict(), "recommender_model")


# Evaluation

model.eval()
predictions = []
with torch.no_grad():
    for batch_idx, (user, item, rating) in enumerate(dataloader):
        user, item = user.to(device), item.to(device)
        prediction = model(user - 1, item - 1)
        predictions.extend(prediction.view(-1).tolist())


  0%|          | 0/12 [00:00<?, ?it/s]

Epoch 1/12, Avg Loss: 1.1189
Epoch 2/12, Avg Loss: 0.8982
Epoch 3/12, Avg Loss: 0.8667
Epoch 4/12, Avg Loss: 0.8503
Epoch 5/12, Avg Loss: 0.8382
Epoch 6/12, Avg Loss: 0.8271
Epoch 7/12, Avg Loss: 0.8180
Epoch 8/12, Avg Loss: 0.8089
Epoch 9/12, Avg Loss: 0.8003
Epoch 10/12, Avg Loss: 0.7928
Epoch 11/12, Avg Loss: 0.7844
Epoch 12/12, Avg Loss: 0.7772


In [65]:
# Example

sample_user_id = 1
sample_user_data = ratings[ratings['user_id'] == sample_user_id]
sample_dataset = RatingsDataset(sample_user_data)
sample_dataloader = DataLoader(sample_dataset, batch_size=1, shuffle=False)

print(f'Sample predictions for user ID {sample_user_id}:')
with torch.no_grad():
    for user, movie, rating in sample_dataloader:
        output = model(user, movie).squeeze()
        print(f'Movie ID: {movie.item()}, Predicted Rating: {output.item()}, Actual Rating: {rating.item()}')

Sample predictions for user ID 1:
Movie ID: 1193, Predicted Rating: 3.5492844581604004, Actual Rating: 5.0
Movie ID: 661, Predicted Rating: 3.151799440383911, Actual Rating: 3.0
Movie ID: 914, Predicted Rating: 4.080698013305664, Actual Rating: 3.0
Movie ID: 3408, Predicted Rating: 3.357598066329956, Actual Rating: 4.0
Movie ID: 2355, Predicted Rating: 3.2987260818481445, Actual Rating: 5.0
Movie ID: 1197, Predicted Rating: 4.3291850090026855, Actual Rating: 3.0
Movie ID: 1287, Predicted Rating: 4.143976211547852, Actual Rating: 5.0
Movie ID: 2804, Predicted Rating: 2.9534459114074707, Actual Rating: 5.0
Movie ID: 594, Predicted Rating: 3.825355291366577, Actual Rating: 4.0
Movie ID: 919, Predicted Rating: 3.925562858581543, Actual Rating: 4.0
Movie ID: 595, Predicted Rating: 3.7510018348693848, Actual Rating: 5.0
Movie ID: 938, Predicted Rating: 3.4220008850097656, Actual Rating: 4.0
Movie ID: 2398, Predicted Rating: 2.9412825107574463, Actual Rating: 4.0
Movie ID: 2918, Predicted Rat

In [67]:
model.to('cpu')
def predict_unrated_movies(user_id, model, ratings, movies):
    #TODO

    user_data = ratings[ratings['user_id'] == user_id]
    user_movies = set(user_data['movie_id'])

    unrated_movies = movies[~movies['movie_id'].isin(user_movies)]
    unrated_movie_ids = unrated_movies['movie_id'].values

    num_unrated_movies = len(unrated_movie_ids)
    user_ids = torch.full((num_unrated_movies,), user_id, dtype=torch.long)
    movie_ids = torch.tensor(unrated_movie_ids, dtype=torch.long)

    with torch.no_grad():
        predictions = model(user_ids - 1, movie_ids - 1).squeeze().tolist()

    predictions_df = pd.DataFrame({
        'movie_id': unrated_movie_ids,
        'predicted_rating': predictions
    })

    predicted_movies = pd.merge(movies, predictions_df, on='movie_id')

    predicted_movies_sorted = predicted_movies.sort_values('predicted_rating', ascending=False)

    return predicted_movies_sorted


# Example
user_id = 1
predictions = predict_unrated_movies(user_id, model, ratings, movies)
print(predictions)

      movie_id                                title                   genres  \
2904      3022                  General, The (1927)               ['Comedy']   
2392      2503              Apple, The (Sib) (1998)                ['Drama']   
2788      2905                       Sanjuro (1962)  ['Action', 'Adventure']   
3124      3245  I Am Cuba (Soy Cuba/Ya Kuba) (1964)                ['Drama']   
763        787   Gate of Heavenly Peace, The (1995)          ['Documentary']   
...        ...                                  ...                      ...   
2682      2798                 Problem Child (1990)               ['Comedy']   
2444      2555                 Baby Geniuses (1999)               ['Comedy']   
2683      2799               Problem Child 2 (1991)               ['Comedy']   
188        193                     Showgirls (1995)                ['Drama']   
2701      2818                 Iron Eagle IV (1995)        ['Action', 'War']   

      predicted_rating  
2904          

### GMM (Gaussian Mixture Model)

**Objective**: Use a Gaussian Mixture Model to analyze and cluster the click data based on the number of clicks from different locations, aiming to identify distinct patterns of user behavior across 10 countries.

**Dataset**:
- **Click Data**: DataFrame with columns `link_id`, `location`, and `number_of_clicks`.

**Steps**:

1. **Preprocess Data**:
   - Ensure the dataset contains 10 distinct countries.
   - Create a matrix where rows represent different links and columns represent the number of clicks from each country.
   - Normalize the number of clicks to account for different scales.

2. **Fit GMM**:
   - Apply a Gaussian Mixture Model (GMM) to the click data matrix to identify clusters of links with similar click patterns across different countries.

3. **Evaluate Model**:
   - Analyze the resulting model by calculating the log lokelihood, BIC and AIC metrics.


In [68]:
import pandas as pd
import numpy as np
from sklearn.mixture import GaussianMixture
import matplotlib.pyplot as plt

file_path = '/content/drive/MyDrive/IR/gmm-dataset.csv'
df = pd.read_csv(file_path)

df_standardized = (df.iloc[:, 1:] - df.iloc[:, 1:].mean()) / df.iloc[:, 1:].std()

n_components = 10

gmm = GaussianMixture(n_components=n_components)
gmm.fit(df_standardized)

clusters = gmm.predict(df_standardized)
df['cluster'] = clusters

log_likelihood = gmm.score(df_standardized)
bic = gmm.bic(df_standardized)
aic = gmm.aic(df_standardized)

print(f"Log-Likelihood: {log_likelihood}")
print(f"BIC: {bic}")
print(f"AIC: {aic}")

Log-Likelihood: -1.430761925500829
BIC: 3320.959537666318
AIC: 1604.1523851001657
